In [1]:
import zipfile
with zipfile.ZipFile("adata_covid19_haniffa_red_final_overlap_4209_RELABELED.h5ad.zip", 'r') as zip_ref:
    zip_ref.extractall("train")

In [3]:
import xgboost as xgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
!pip install scanpy
import scanpy
import numpy as np
import os
import pandas as pd
import gc

     |████████████████████████████████| 2.0 MB 4.0 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 91 kB 9.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82709 sha256=a59f6949c3ae1c19adea03bc850f37aeeb465f50096725c87b2726a483e2a89d
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52603 sha256=98714c4208057f45502bfbe1876f9ed158a42b6610c622c16db140c84b1039b0
  Stored in directory: /root/.cache/pip/wheels/af/e9/33/04db1436df0757c42fda8ea6796d7a8586e23c85fac355f476
  Created wheel for sinfo: filename=sinfo-0.3.4-py3-none-any.whl size=7899 sha256=d3e37ea4008b9616f69db11fc8777de0709df6935d467d5a9cca33d408998542
  Stored in directory: /root/.cache/pi

In [4]:
!pip install anndata
from anndata import read_h5ad

In [5]:
df = read_h5ad("train/adata_covid19_haniffa_red_final_overlap_4209_RELABELED.h5ad")
scanpy.pp.normalize_total(df)
scanpy.pp.log1p(df)
X_train = df.X
y_train = df.obs["new_cell_label"]

In [ ]:
"""
#!pip install lightgbm
import lightgbm as lgb
train_data = lgb.Dataset(X_train,label=y_train)
param = {'num_iterations':100,'metric':'multi_logloss','objective':'multiclass','num_class':15}
model = lgb.train(params=param,train_set=train_data)
"""

In [6]:
del df
gc.collect()
model = XGBClassifier()
model.fit(X_train, y_train)
print("fitting done")

fitting done


In [7]:
model.save_model("model.txt")
del X_train
del y_train
gc.collect()

807

In [8]:
with zipfile.ZipFile("adata_covid19_test_combined_overlap_4209.h5ad.zip", 'r') as zip_ref:
    zip_ref.extractall("test")

In [9]:
df = read_h5ad("test/adata_covid19_test_combined_overlap_4209.h5ad")
scanpy.pp.normalize_total(df)
scanpy.pp.log1p(df)
X_test = df.X
y_test = df.obs["new_cell_label"]

In [10]:
y_pred = model.predict(X_test)
print("predictions done")
del X_test
gc.collect()
#predictions = [round(value) for value in y_pred]
#del y_pred
#gc.collect()
#print("rounded up")
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

predictions done
Accuracy: 5.41%
